# Big data lab

## import library

In [1]:
import pandas as pd
import pickle
import my_func
import lightgbm as lgb
file_name = 'churn_test_model_fe.pickle'
file_path = r'D:\BDL-Final\data-from-BDL' + '\\'

# settings for dataframe
pd.set_option('display.max_columns', None)

In [19]:
df_fe = pd.read_pickle(file_path+file_name)

# Exploratory Data Analysis (EDA)

## First look

In [13]:
#df_fe.head(5)

In [22]:
df_fe.describe()

,Ama_rchrgmnt_sum_max_mnt1,content_clc_mea_mnt1,content_cnt_max_mnt1,voice_out_short_part_max_mnt1,voice_mts_in_nrest_part_std_mnt1,num_act_days_max_mnt1,sms_roam_clc_min_mnt1,voice_in_cmpttrs_avg_durmin_mnt1,com_num_part_mea_mnt1,pay_avg_mea_mnt1,voice_out_tar_dur_std_mnt1,voice_out_tar_dur_min_mnt1,gprs_clc_mea_mnt1,pay_max_min_mnt3,voice_in_short_part_mea_mnt3,voice_in_roam_clc_std_mnt1,conn_out_uniq_cnt_std_mnt1,voice_mts_in_dwork_part_td_mnt3,data_3g_tar_vol_mea_mnt3,voice_in_fix_tar_dur_mea_mnt3,voice_in_tar_dur_max_mnt3,ks_num_part_td_mnt3,data_3g_dou_mea_mnt3,voice_mts_out_drest_partmea_mnt1,conn_com_part_min_mnt1,Ama_rchrgmnt_sum_mea_mnt3,voice_mts_in_dwork_part_mea_wk1,voice_mts_in_dwork_part_std_mnt1,gprs_clc_std_mnt1,conn_com_part_td_mnt3,sms_clc_min_mnt3,gprs_clc_mea_mnt3,gprs_clc_min_mnt3,voice_in_cmpttrs_avg_durmax_mnt3,voice_in_tar_dur_td_mnt3,conn_in_uniq_cnt_td_mnt3,voice_mts_out_nrest_partmax_mnt1,voice_in_tar_dur_std_mnt1,voice_mts_out_dwork_partmea_wk1,voice_out_fix_tar_dur_td_mnt3,all_clc_mea_mnt1,voice_out_cmpttrs_td_cnttd_mnt3,all_roam_clc_mea_mnt1,voice_in_tar_dur_std_mnt3,voice_in_td_cnt_max_mnt3,voice_out_fix_tar_dur_mea_wk1,conn_com_part_std_mnt3,voice_in_roam_clc_mea_mnt3,pay_p2p_in_sum_max_mnt3,content_cnt_mea_mnt1,voice_in_roam_clc_mea_wk1,com_num_part_min_mnt1,pay_p2p_in_sum_mea_wk1,all_home_clc_td_mnt3,voice_out_td_cnt_mea_mnt1,vas_clc_mea_wk1,com_num_part_max_mnt1,voice_mts_out_drest_partstd_mnt3,pay_sum_td_mnt3,pay_p2p_in_sum_std_mnt1,conn_out_uniq_cnt_std_mnt3,all_clc_min_mnt3,all_roam_clc_min_mnt3,pay_p2p_out_sum_max_mnt3,voice_in_cmpttrs_td_cnt_max_mnt1,voice_mts_out_nrest_partstd_mnt3,voice_in_td_cnt_mea_mnt3,data_3g_dou_min_mnt3,voice_mts_in_nrest_part_mea_wk1,voice_in_fix_tar_dur_max_mnt1,ks_num_part_mea_wk1,pay_p2p_out_sum_std_mnt1,block_all_dur_min_mnt1,voice_in_cmpttrs_td_cnt_max_mnt3,pay_sum_std_mnt1,abon_part_max_mnt3,sms_out_cnt_std_mnt3,sms_in_cnt_min_mnt1,conn_in_uniq_cnt_mea_wk1,pay_sum_min_mnt3,ama_volume_max_mnt3,vas_clc_min_mnt3,voice_in_td_cnt_std_mnt1,sms_roam_clc_max_mnt1,all_cnt_min_mnt1,voice_in_short_part_max_mnt1,accum_mts_dur_std_mnt3,data_3g_tv_cnt_mea_wk1,voice_out_td_cnt_min_mnt1,data_3g_tar_vol_mea_mnt1,voice_mts_out_nrest_partmin_mnt1,all_cnt_mea_mnt1,data_3g_dou_std_mnt3,voice_in_short_part_min_mnt3,sms_in_cnt_td_mnt3,voice_mts_out_dwork_partstd_mnt1,voice_out_short_part_mea_wk1,voice_in_fix_tar_dur_mea_wk1,block_all_dur_min_mnt3,pay_sum_max_mnt1,content_cnt_mea_wk1,voice_mts_out_nwork_partstd_mnt1,com_num_part_mea_mnt3,conn_out_uniq_cnt_max_mnt3,content_clc_mea_mnt3,Ama_rchrgmnt_sum_min_mnt3,abon_part_min_mnt3,conn_out_uniq_cnt_mea_mnt3,gprs_clc_std_mnt3,voice_out_short_part_min_mnt1,all_cnt_std_mnt1,conn_com_part_mea_wk1,com_num_part_max_mnt3,voice_out_fix_tar_dur_max_mnt1,pay_p2p_out_sum_mea_mnt1,abon_part_min_mnt1,accum_mts_dur_min_mnt1,content_clc_min_mnt1,voice_out_fix_tar_dur_std_mnt1,voice_mts_out_nwork_partmin_mnt1,vas_clc_max_mnt3,sms_roam_clc_mea_wk1,pay_avg_std_mnt1,content_clc_min_mnt3,pay_p2p_out_sum_mea_mnt3,data_3g_tar_vol_std_mnt1,vas_clc_min_mnt1,sms_in_cnt_mea_mnt1,goodok_clc_max_mnt3,com_num_part_td_mnt3,all_home_clc_mea_mnt3,voice_in_short_part_max_mnt3,pay_max_max_mnt1,sms_clc_std_mnt1,pay_max_mea_mnt1,accum_mts_dur_td_mnt3,gprs_clc_max_mnt3,vas_clc_mea_mnt1,all_clc_mea_mnt3,voice_in_kievstar_part_min_mnt1,Ama_rchrgmnt_sum_mea_wk1,sms_in_cnt_min_mnt3,voice_mts_in_drest_part_td_mnt3,voice_mts_in_dwork_part_min_mnt1,voice_in_cmpttrs_td_cnt_mea_mnt1,sms_roam_clc_max_mnt3,pay_max_mea_mnt3,gprs_clc_td_mnt3,ks_num_part_std_mnt1,voice_in_mts_avg_dur_max_mnt1,voice_in_td_cnt_mea_mnt1,voice_mts_in_drest_part_max_mnt3,accum_mts_dur_max_mnt3,pay_p2p_out_sum_max_mnt1,sms_roam_clc_std_mnt3,voice_out_cmpttrs_td_cntmin_mnt3,voice_out_cmpttrs_avg_dumax_mnt1,voice_mts_in_nrest_part_mea_mnt3,voice_out_tar_dur_max_mnt1,voice_mts_in_drest_part_std_mnt1,clc_no_vas_roam_std_mnt3,non_accum_internet_vol_mea_wk1,voice_mts_out_nwork_partmea_mnt1,voice_mts_in_nrest_

In [20]:
cat_info = my_func.categorical_col_info(df_fe)

In [21]:
cat_info

,Column,Unique values,Missing values
Total,0,0,0


# Feature Engineering
